In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import re
from tensorflow import keras
from tensorflow.keras import layers

In [2]:
# read text data from data_list.txt
my_file = open("data_list_short.txt", "r")
path_data = my_file.read()
paths = path_data.split("\n")
print(paths)
my_file.close()

['ShortStories/An_Occurance_at_Owl_Creek_Bridge.txt', 'ShortStories/Bartleby, the Scrivener.txt', 'ShortStories/Rip_Van_Winkle.txt', 'ShortStories/Tell_Tale_Heart.txt', 'ShortStories/The_Last_Question.txt', 'ShortStories/The_Lottery.txt', 'ShortStories/The_Monkeys_Paw.txt', 'ShortStories/The_Most_Dangerous_Game.txt', 'ShortStories/The_Turn_of_the_Screw.txt', 'ShortStories/The_Yellow_Wallpaper.txt']


In [3]:
contents = []

for path in paths:
    with open(path, 'r', encoding='utf-8') as f:  # encoding='utf-8'不行
        for line in f.readlines():
            contents.append(line)
    f.close()

In [18]:
# read rate scores from rate.txt
rate = []
rate_str = []
my_file = open("rate_short.txt", "r")
rate_data = my_file.read()
rate_str = rate_data.split("\n")
for score in rate_str:
    rate.append(float(score))
print(rate)
my_file.close()

# lb = LabelEncoder()
# rate = lb.fit_transform(rate)
type(rate[0])

[2.56, 4.1, 3.69, 4.3, 4.59, 4.04, 3.78, 4.15, 3.34, 3.69]


float

In [19]:
# build data frame ("test", "label")
data = pd.DataFrame([])
data["text"] = contents
data["label"] = rate
print(data)

                                                text  label
0  ﻿A man stood upon a railroad bridge in norther...   2.56
1  ﻿A Story of Wall-Street  I AM A rather elderly...   4.10
2  ﻿Whoever has made a voyage up the Hudson must ...   3.69
3  ﻿TRUE!—NERVOUS—VERY, very dreadfully nervous I...   4.30
4  ﻿The last question was asked for the first tim...   4.59
5  ﻿The morning of June 27th was clear and sunny,...   4.04
6  ﻿PART ONE Outside, the night was cold and wet,...   3.78
7  ﻿The Most Dangerous Game “OFF THERE to the rig...   4.15
8  ﻿The story had held us, round the fire, suffic...   3.34
9  ﻿It is very seldom that mere ordinary people l...   3.69


In [20]:
# tokenize the text data
token = re.compile('[A-Za-z]+|[!?,.()]')
def reg_text(text):
    new_text = token.findall(text)
    new_text = [word.lower() for word in new_text]
    return new_text

data['text'] = data.text.apply(reg_text)
word_set = set()
for text in data.text:
    for word in text:
        word_set.add(word)

In [21]:
# make dataset for training, make sure the size of each book's vector is the same
max_word = len(word_set) + 1
word_list = list(word_set)

word_index = dict((word, word_list.index(word) + 1) for word in word_list)
data_ok = data.text.apply(lambda x: [word_index.get(word, 0) for word in x])

maxlen = max(len(x) for x in data_ok)
data_ok = tf.keras.preprocessing.sequence.pad_sequences(data_ok.values, maxlen=maxlen)
print("Shape of data_ok: ", data_ok.shape)

Shape of data_ok:  (10, 49644)


In [33]:
# build ther model and train
def train_model():
    model = keras.Sequential()
    model.add(layers.Embedding(max_word, 16, input_length=maxlen))
    model.add(layers.Bidirectional(layers.LSTM(64,
                         dropout=0.2,
                         recurrent_dropout=0.5)))
    model.add(layers.Dropout(0.5))
    model.add(layers.Dense(1, activation='sigmoid'))
    model.compile(optimizer=keras.optimizers.RMSprop(),
              loss='binary_crossentropy',
              metrics=['acc'])
    return model

In [34]:
model = train_model()
print(model.summary())

Model: "sequential_6"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 49644, 16)         139552    
_________________________________________________________________
bidirectional_7 (Bidirection (None, 128)               41472     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 181,153
Trainable params: 181,153
Non-trainable params: 0
_________________________________________________________________
None


In [24]:
learning_rate_reduction = keras.callbacks.ReduceLROnPlateau(monitor='val_loss', patience=3, factor=0.3, min_lr=0.00001)
history = model.fit(data_ok,
                     data.label.values,
                     epochs=50,
                     batch_size=128,
                     validation_split=0.2,
                     callbacks=[learning_rate_reduction])

Epoch 1/2
1/1 [==============================] - 3s 3s/step - loss: 0.6811 - acc: 0.0000e+00 - val_loss: 0.5250 - val_acc: 0.0000e+00
Epoch 2/2
1/1 [==============================] - 3s 3s/step - loss: 0.4487 - acc: 0.0000e+00 - val_loss: 0.3624 - val_acc: 0.0000e+00


In [11]:

# save the model as model1.keras
model.save('model/model1.keras')